<div style="text-align: right" align="right"><i>Peter Norvig<br> Feb 2024</i></div> 

# Probabilities for Overtime in the Super Bowl

In American football, if the game is tied at the end of 60 minutes of play, an overtime period commences, with these rules:

- There is a coin toss; the team that wins the flip has the option of possessing the ball first or second.
- Both teams get one possession to attempt to score points.
- After those two possessions, if the score is still tied, the game continues and the next score wins the game.

A recap of (most of) the scoring rules:

- A team scores **3** points for kicking a **field goal**. 
- A team scores **6** points for a **touchdown**. 
- After scoring a touchdown, if the game is not over, a team gets a chance to score **extra point**(s); they have the choice of going for either **one** point (easier) or **two** points (harder).



The 2024 Super Bowl went into overtime, and there was some criticism of San Francisco 49ers coach Kyle Shanahan, who, after winning the coin toss, elected to possess the ball first rather than second. As it turned out the 49ers scored a field goal and then the Chiefs scored a touchdown to win. If the 49ers had taken the ball second, and they had known that the Chiefs scored a touchdown first, they could have gone for their own touchdown rather than the field goal, and perhaps tied or won the game. There are two key question:

1) Is it better to possess the ball first  or second?
2) After a team scores a touchdown, should they go for 1 extra point or 2?
# TL;DR

The short answers:
1) It is (slightly) better to **take the ball first.** (Why? Despite the avantage of possessing the ball second rather than first, it is more important to possess it *third*, because then if you score you win. So it is better to be first (and myabe third) than second (and maybe fourth).)
3) The first team should **go for 1** extra point. If they succeed, the second team should **go for 2** extra points. 

In this notebook I make some simplifying assumptions, and then do a simulation  to verify these answers.


## Code to Simulate One Random Overtime Game



Calling the function `overtime()` below runs a random simulation of an overtime game and returns a pair of integers, (*A*, *B*), indicating the scores for the first team to possess the ball (**A**) and the second team (**B**). I could have done a detailed play-by-play simulation, but I decided to use a possession-by-possession simulation that considers just a few probabilities (and makes the inaccurate assumption that each possession is independent of the previous ones). You can set the following probability parameters (they will be the same for both teams):
- **TD**: the probability of scoring a touchdown on a given possession. (By default 20%, the league average in recent years.)
- **FG**: the probability of scoring a field goal on a given possession. (By default 25%.)
- **go**: the *additional* probability of scoring a touchdown for a team that must score a touchdown or lose–i.e., if team **A** scores a touchdown on their first possession, team **B** would never attempt a field goal (or punt). They will instead *go* for a touchdown, which they get with probability **go**. (By default 5%, but I'm not sure if that is a good estimate.)
- **one**: the probability of succesfully kicking a 1-point conversion attempt. (About 98%.)
- **two**: the probability of succesfully scoring on a 2-point conversion attempt. (About 48%.)
- **A_extra**: 1 or 2, denoting what **A** should try for if they score a touchdown on their first possession.
- **B_extra**: 1 or 2, denoting what **B** should try for if **A** scored 7 points on their first possession.


In [1]:
import random
from collections import Counter
from typing import Iterable, Tuple
from statistics import mean

Prob = float # The type for a probability, a number between 0 and 1.
Scores = Tuple[int, int] # Type for an (A, B) score.

def overtime(TD=0.20, FG=0.25, go=0.05, one=0.98, two=0.48, A_extra=1, B_extra=2) -> Scores:
    """Given probabilities, play a random overtime and return (team_A_score, team_B_score).
    Both teams have same probabilities:
    TD:  probability of scoring a touchdown on a 'normal' possession. 
    FG:  probability of scoring a field goal on a 'normal' possession.
    go:  additional probability of scoring a touchdown, if you must score or lose.
    one: probability of making a one-point conversion.
    two: probability of making a two-point conversion.
    A_extra: what team A goes for on the extra point (1 or 2).
    B_extra: what team B goes for on the extra point (1 or 2), when behind by 1."""
    A = B = 0 # The scores of the two teams
    first_possession = True
    while A == B:
        extra = (0 if not first_possession else P(one, 1) if A_extra == 1 else P(two, 2))
        A += score(TD, FG, extra)
        if first_possession or A == B: # B gets a chance on their first possession, or if the game is still tied.
            extra = (0 if B + 6 > A else P(one, 1) if B + 7 > A or (B_extra == 1 and B + 7 == A) else P(two, 2))
            B += (score(TD, FG, 0) if B + 3 >= A else score(TD + go, 0, extra)) # must go for it if down by more than 3
        first_possession = False
    return A, B # The scores of the two teams

def score(TD: Prob, FG: Prob, extra: int) -> int: 
    """Randomly simulate a score, given probabilities for TD and FG, and given the number of extra points."""
    return P(TD, 6 + extra) or P(FG / (1 - TD), 3)

def P(p: Prob, points: int) -> int: 
    """Return `points` with probability `p`, else 0."""
    return points if random.random() < p else 0

Let's play a random overtime game and see the scores of the two teams:

In [2]:
overtime()

(0, 3)

# Code to Simulate a Million Games, and Draw Conclusions

OK, but that's just one game. What if we play, say, a million games?

In [3]:
def overtimes(games=1_000_000, **kwds) -> Iterable[Scores]:
    """Play `games` games and return an iterable of the `(A, B)` score for each game."""
    return (overtime(**kwds) for _ in range(games))

Counter(overtimes()).most_common()

[((3, 0), 246591),
 ((0, 3), 198249),
 ((0, 6), 157443),
 ((7, 0), 146508),
 ((6, 0), 89334),
 ((3, 6), 62197),
 ((7, 6), 25421),
 ((7, 8), 23293),
 ((6, 3), 22070),
 ((9, 3), 18140),
 ((3, 9), 9734),
 ((6, 7), 996),
 ((12, 6), 9),
 ((9, 6), 8),
 ((6, 12), 4),
 ((6, 9), 3)]

That gives us the  possible scores and the frequency of each one. Note:
- Field goals are more common than touchdowns, so the most common scores are 3-0 and 0-3.
- 3-0 is more common than 0-3, because 3-0 includes times when **A** scored first and also third (or later, but that's rarer).
- Why is 0-6 more common than 7-0? Because 0-6 includes times when **B** scored second or fourth (or later), but 7-0 is only first.
- Note that the total of 7-0 (**A** scores first) and 6-0 (**A** scores third, or first and misses the kick) is more than the count for 0-6.
- Note that 3-6 is almost 3 times more common than 6-3. Either could result from 3 field goals, but 3-6 could also be **B** scoring a touchdown; 6-3 could never result from **A** scoring a touchdown.
- 6-9, 9-6, 12-6, and 6-12 are very rare scores, because they mean that *both* teams missed a conversion kick.

That's all interesting, but the two questions remain:

# Question 1: Who Has the Advantage, Team A or Team B?

We can acompute the probability of team **A** winning:

In [4]:
def A_win_probability(scores) -> float:
    """Probability that first team (team A) wins."""
    return mean(A > B for A, B in scores)

In [5]:
A_win_probability(overtimes())

0.548365

My simulation says team **A** has nearly a 55% chance of winning. This supports Shanahan's decision. 


# Question 2: What Are the Best Strategies for Extra Points?

Earlier I claimed that **A** should go for 1 extra point and **B** for 2:

In [6]:
A_win_probability(overtimes(A_extra=1, B_extra=2))

0.548867

We can see that if **A** goes for 2, they will do worse (by about half a percent):

In [7]:
A_win_probability(overtimes(A_extra=2, B_extra=2))

0.54451

We can also see that if **B** goes for 1, they will do worse (i.e., **A** will do better):

In [8]:
A_win_probability(overtimes(A_extra=1, B_extra=1))

0.555354

These results support the claim that **A** should go for 1 extra point and **B** for 2.

# Question 1: Varying the Parameters

However, I don't have confidence that the probability parameter values I chose are reflective of reality, so for each parameter I'll try a lower and a higher value, and look at all combinations, displayed as a dataframe:

In [9]:
import pandas as pd
from IPython.display import HTML
    
def chart(TDs=(0.15, 0.20, 0.33), 
          FGs=(0.20, 0.25, 0.33), 
          gos=(0.02, 0.05, 0.10), 
          twos=(0.4, 0.48, 0.60),
          games=100_000) -> HTML:
    """Create a chart of Win percentages for various parameter values."""
    data = [(TD, FG, go, two, A_win_probability(overtimes(games=games, TD=TD, FG=FG, go=go)))
            for TD in TDs for FG in FGs for go in gos for two in twos]
    df = pd.DataFrame(data, columns=('TD', 'FG', 'go', 'two', 'Win')).sort_values('Win', ascending=False)
    print(f'Team A win probability: min: {min(df.Win):.1%}, max: {max(df.Win):.1%}')
    return HTML(df.to_html(index=False, formatters={'Win': '{:.1%}'.format}))

In [10]:
chart()

Team A win probability: min: 53.0%, max: 56.1%


TD,FG,go,two,Win
0.15,0.33,0.02,0.48,56.1%
0.15,0.33,0.02,0.40,56.0%
0.20,0.33,0.02,0.48,55.8%
0.15,0.33,0.02,0.60,55.7%
0.15,0.33,0.05,0.40,55.7%
0.20,0.33,0.02,0.40,55.7%
0.15,0.33,0.05,0.60,55.6%
0.20,0.33,0.02,0.60,55.6%
0.20,0.33,0.05,0.60,55.5%
0.20,0.33,0.05,0.40,55.5%


The results show that over a wide range of assumptions about the probabilities, team **A** always wins a majority of the time (between 53% and 56%). So no more Shanahan shaming.